In [ ]:
import re
import statistics as stat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
# scrapping_with_selenium: past project which scrapes data from wuzzuf(check it in my past repo)
import scrapping_with_selenium as sc


In [ ]:
# the function which scrapes the data for the first  pages
jobs = ["analyst",
        "Data analysis",
        "Data Analyst",
        "Business Analyst",
        "Data Engineer",
        "Data Scientist",
        "Bi developer",
        "Business Intelligence"]

table = sc.scraper("Data", 150, jobs, 0.7)

In [ ]:
table.to_csv("D:\Projects\Data Roles Market\df.csv", index=False)


In [10]:
# read the csv file
df = pd.read_csv("D:\Projects\Data Roles Market\df.csv")
df.head()


,job_id,job_title,company,location,experience,salaries,skill,job_type,open_postions
0,10,Data Analyst,Confidential -,"Heliopolis, Cairo, Egypt",· 2 - 6 Yrs of Exp,Confidential,analytical,Full Time,3
1,10,Data Analyst,Confidential -,"Heliopolis, Cairo, Egypt",· 2 - 6 Yrs of Exp,Confidential,Analyst,Full Time,3
2,10,Data Analyst,Confidential -,"Heliopolis, Cairo, Egypt",· 2 - 6 Yrs of Exp,Confidential,Data,Full Time,3
3,10,Data Analyst,Confidential -,"Heliopolis, Cairo, Egypt",· 2 - 6 Yrs of Exp,Confidential,Data Analyst,Full Time,3
4,11,Data Management Engineer,Ejada -,"Cairo, Egypt",· 3 - 5 Yrs of Exp,Confidential,Computer Science,Full Time,4


In [11]:
# only keep job titles that contains(Data, Analyst & Bi)
df2 = df.copy()
df2 = df2[
    df2["job_title"].str.contains("Data") |
    df2["job_title"].str.contains("Analyst") |
    df2["job_title"].str.contains("BI") |
    df2["job_title"].str.contains("Business Intelligence")
]


In [12]:
# create a list of our conditions
conditions = [
    df2["job_title"].str.contains(
        "Analyst") | df2["job_title"].str.contains("Analysis"),
    df2["job_title"].str.contains("BI"),
    df2["job_title"].str.contains("Engineer"),
    df2["job_title"].str.contains("Scientist")
]

# create a list of the values we want to assign for each condition
values = ['data analyst', 'bi developer', 'data engineer', 'data scientist']

# create a new column and use np.select to assign values to it using our lists as arguments
df2['roles'] = np.select(conditions, values)
df2 = df2[df2.roles.isin(values) == True]


In [13]:
# extract cities only
df2["location"] = df2["location"].map(lambda x: x.split(',')[0])

# clean companies names & extract experience range
df2["company"] = df2["company"].str.replace(" -", "")
df2["experience"] = df2["experience"].str.replace(" ", "")
df2["experience"] = df2["experience"].str.findall(".(.+?)Y")
df2["experience"] = [''.join(map(str, l)) for l in df2["experience"]]


In [14]:
conditions = [
    df2["experience"].str.contains("^0.*"),
    df2["experience"].str.contains("^1.*"),
    df2["experience"].str.contains("^[23].*"),
    df2["experience"].str.contains("^[45].*"),
    df2["experience"].str.contains("^[67].*")
]

values = ['entry level', 'junior', 'mid level', 'senior', 'team lead']

df2['levels'] = np.select(conditions, values)


In [15]:
def clean_salaries(df, rol, lst):

    conds = [
        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'entry level'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'junior'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'mid level'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'senior'),

        (df['roles'] == rol) &
        ((df["salaries"].str.contains("Confidential")) | (df["salaries"].str.contains("Male")) | (df["salaries"].str.contains("Females"))) &
        (df['levels'] == 'team lead')
    ]

    for _ in range(len(conds)):
        df['salaries'] = np.where(conds[_], lst[_], df["salaries"])


In [16]:
# replace confidential with avg market salaries lists
sal = {
    "data analyst": [6000, 8000, 10950, 28500, 28500],
    "bi developer": [5625, 8550, 10000, 15000, 20000],
    "data engineer": [9000, 10375, 23000, 30000, 47000],
    "data scientist": [9400, 11000, 19200, 39584, 67815]
}

for i in sal:
    clean_salaries(df2, i, sal[i])


In [17]:
# replace ("number" To "number") expression, to only mean value
df2["salaries"] = df2["salaries"].apply(lambda salary: re.findall(
    (r"\d+"), salary) if type(salary) == str else salary)

df2["salaries"] = df2["salaries"].apply(lambda salary: list(
    map(int, salary)) if type(salary) == list else salary)

df2["salaries"] = df2["salaries"].apply(
    lambda salary: stat.mean(salary) if type(salary) == list else salary)

df2["salaries"] = df2["salaries"].round()


In [18]:
df_skills = df2.copy()
# continue the list of conditions
conditions = [
    (df_skills["skill"].str.contains("(?i)excel")
     | df_skills["skill"].str.contains("(?i)office")
     | df_skills["skill"].str.contains("(?i)Accounting")),

    (df_skills["skill"].str.contains("(?i)power bi")
     | df_skills["skill"].str.fullmatch("(?i)bi")),

    (df_skills["skill"].str.contains("(?i)sql server")
     | df_skills["skill"].str.fullmatch("(?i)ssis")
     | df_skills["skill"].str.fullmatch("(?i)ssas")
     | df_skills["skill"].str.fullmatch("(?i)ssrs")),

    df_skills["skill"].str.fullmatch("(?i)sql"),

    (df_skills["skill"].str.contains("(?i)computer science")
     | df_skills["skill"].str.fullmatch("(?i)coding")
     | df_skills["skill"].str.fullmatch("(?i)software")),

    df_skills["skill"].str.fullmatch("(?i)Tableau"),

    df_skills["skill"].str.fullmatch("(?i)Statistics"),

    df_skills["skill"].str.fullmatch("(?i)python"),

    df_skills["skill"].str.contains("(?i)spark")
]

values = ['excel', 'power bi', 'sql server', 'sql',
          'programming', 'tableau', 'Statistics', 'python', 'spark']

df_skills['skill'] = np.select(conditions, values)
df_skills = df_skills[df_skills.skill.isin(values) == True]


# EDA